In [ ]:
from bs4 import BeautifulSoup
import requests
import numpy as np
from tabulate import tabulate

In [ ]:
def generate_url(search_string):
    search_string = search_string.strip().replace(" ", "%20")
    url = r'https://www.fiverr.com/search/gigs?query='+ search_string + r'&source=top-bar&search_in=everywhere&search-autocomplete-original-term='+ search_string
    return url

In [ ]:
def get_random_ua():
    random_ua = ''
    ua_file = 'UserAgents.txt'
    try:
        with open(ua_file) as f:
            lines = f.readlines()
        if len(lines) > 0:
            prng = np.random.RandomState()
            index = prng.permutation(len(lines) - 1)
            idx = np.asarray(index, dtype=np.integer)[0]
            random_ua = lines[int(idx)]
    except Exception as ex:
        print('Exception in random_ua')
        print(str(ex))
    finally:
        return random_ua.strip('\n')

In [ ]:
# url = r'https://www.fiverr.com/search/gigs?query=machine%20learning&source=top-bar&search_in=everywhere&search-autocomplete-original-term=machine%20learning'

def get_soup(search_string, print_soup=False):
    headers = {
        'user-agent': get_random_ua(),
        'referrer': r'https://google.com',
        'accept': r'*/*',
        'accept-encoding': r'gzip, deflate, br',
        'accept-language': r'en-US,en;q=0.9,bn;q=0.8',
        'cache-control': r'no-cache',
        #'origin': r'https://www.fiverr.com',
        'pragma': r'no-cache'
    }
    
    url = generate_url(search_string)

    try:
        r = requests.get(url, headers=headers)
        source = r.text
        r.raise_for_status()
    except requests.exceptions.HTTPError as errh:
        print ("Http Error:",errh)
        raise SystemExit(err)
    except requests.exceptions.ConnectionError as errc:
        print ("Error Connecting:",errc)
        raise SystemExit(err)
    except requests.exceptions.Timeout as errt:
        print ("Timeout Error:",errt)
        raise SystemExit(err)
    except requests.exceptions.RequestException as err:
        print ("OOps: Something Else",err)
        raise SystemExit(err)

    soup = BeautifulSoup(source, 'lxml')  # pip install lxml

    if(print_soup is True):
        print(soup.prettify())

    return soup

In [ ]:
def extract_gig_info(gig):
    # gig url
    gig_url = 'https://www.fiverr.com' + gig.contents[0]['href']

    # user info
    seller_information = gig.contents[1].find(class_='seller-name')
    user_id = seller_information.contents[0].contents[-1]
    level = 0
    if len(list(seller_information.contents)) >= 2:
        level = seller_information.contents[1].text.lstrip('Level ').rstrip(' Seller')
    user_info = {'user_id' : user_id,
                 'level' : level }

    # description
    description = gig.contents[2].text.lstrip('I will ')

    # rating
    user_rating = gig.contents[3].text
    stars, votes = user_rating.split('(')
    stars = float(stars)
    votes = int(votes.strip(")"))
    rating = {'stars' : stars,
              'votes' : votes}

    # price starts from
    price = gig.contents[4].text
    price = float(price.split('$', 1)[1])

    gig_info = {'url' : gig_url,
                'user_info' : user_info,
                'description' : description,
                'rating' : rating,
                'price' : price}

    return gig_info

In [ ]:
soup = get_soup(r'Video editing', print_soup=True)

In [ ]:
print(soup.find('div', class_='number-of-results').text)

data = [["ID", "Votes", "Price", "Description", "Level", "Stars"]]
gigs = soup.find('div', class_='listing-container')
for single_gig in gigs.find_all('div', class_='gig-wrapper card'):
    try:
        gig_info = extract_gig_info(single_gig)
        data.append([gig_info['user_info']['user_id'],
                     gig_info['rating']['votes'],
                     gig_info['price'],
                     gig_info['description'],
                     gig_info['user_info']['level'],
                     gig_info['rating']['stars']
                     ])

    except Exception as e:
        print(e)
        print(single_gig.prettify())

print(tabulate(data, headers="firstrow", showindex=True, tablefmt="pretty"))